In [3]:
import os
import shutil
import codecs
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


import pickle

#from pmlb import fetch_data
from sdv.tabular import GaussianCopula
from sdv.tabular import CTGAN
from sdv.tabular import CopulaGAN
from sdv.tabular import TVAE
from sdv.evaluation import evaluate
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",600)

In [4]:
input_file = "./japanese_data/EFn_239999999_t2_2104.txt"


with codecs.open(input_file, "r", "shift-jis", "ignore") as f:
    input_df = pd.read_csv(f,header=0, encoding="shift-jis",sep="\t")

    


In [33]:
input_df_fillna = input_df.fillna(input_df.mean())

In [34]:
input_df_fillna

,施設コード,データ識別番号,退院年月日,入院年月日,データ区分,順序番号,行為明細番号,病院点数マスタコード,レセプト電算コード,解釈番号,診療明細名称,使用量,基準単位,明細点数・金額,円点区分,出来高実績点数,行為明細区分情報,行為点数,行為薬剤料,行為材料料,行為回数,保険者番号,レセプト種別コード,実施年月日,レセプト科区分,診療科区分,医師コード,病棟コード,病棟区分,入外区分,施設タイプ
0,239999999,123456799,20210429,20210415,13,1,0,3,120001610,B0082,薬剤管理指導料２（１の患者以外の患者） ...,0.0,0,0.0,0,0,2.152659e+09,325,0,0,1,39231089.0,1317.0,20210415,5.0,340,,11W,0,0.0,NaN
1,239999999,123456799,20210429,20210415,13,1,1,3,120001610,B0082,薬剤管理指導料２（１の患者以外の患者） ...,0.0,0,325.0,0,325,1.000000e+07,0,0,0,1,39231089.0,1317.0,20210415,5.0,340,,11W,0,0.0,NaN
2,239999999,123456799,20210429,20210415,13,2,0,15,113010810,B0081,薬剤管理指導料１（安全管理を要する医薬品投与患者） ...,0.0,0,0.0,0,0,2.152659e+09,380,0,0,1,39231089.0,1317.0,20210422,5.0,340,,11W,0,0.0,NaN
3,239999999,123456799,20210429,20210415,13,2,1,15,113010810,B0081,薬剤管理指導料１（安全管理を要する医薬品投与患者） ...,0.0,0,380.0,0,380,1.000000e+07,0,0,0,1,39231089.0,1317.0,20210422,5.0,340,,11W,0,0.0,NaN
4,239999999,123456799,20210429,20210415,13,2,2,9730,830100078,,タグリッソ ...,0.0,0,0.0,0,0,0.000000e+00,0,0,0,1,39231089.0,1317.0,20210422,5.0,340,,11W,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,239999999,123456799,20210429,20210415,97,28,1,98404,197000110,,食事療養費Ｉ ...,3.0,128,1920.0,1,1920,1.000000e+07,0,0,0,1,39231089.0,1317.0,20210427,5.0,340,,11W,0,0.0,NaN
599,239999999,123456799,20210429,20210415,97,29,0,98404,197000110,,食事療養費Ｉ ...,0.0,0,0.0,1,0,2.152659e+09,1920,0,0,1,39231089.0,1317.0,20210428,5.0,340,,11W,0,0.0,NaN
600,239999999,123456799,20210429,20210415,97,29,1,98404,197000110,,食事療養費Ｉ ...,3.0,128,1920.0,1,1920,1.000000e+07,0,0,0,1,39231089.0,1317.0,20210428,5.0,340,,11W,0,0.0,NaN
601,239999999,123456799,20210429,20210415,97,30,0,98404,197000110,,食事療養費Ｉ ...,0.0,0,0.0,1,0,2.152659e+09,640,0,0,1,39231089.0,1317.0,20210429,5.0,340,,11W,0,0.0,NaN


In [35]:
models = [GaussianCopula(),CTGAN(),CopulaGAN(),TVAE()]

In [36]:
for i in range(len(models)):
    models[i].fit(input_df)

/home/mamitsu/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/mamitsu/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/home/mamitsu/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/home/mamitsu/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:5311: RuntimeWarning: divide by zero encountered in power
  return cd2*x**(c-1)
/home/mamitsu/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measu

In [30]:
row_num = input_df.shape[0]
models[2].sample(1000).iloc[:,10:]

Error: Sampling terminated. Partial results are stored in /home/mamitsu/Github/synthetic_verification/aaa.


ValueError: Cannot convert non-finite values (NA or inf) to integer

In [38]:
new_dfs = []
for i in range(len(models)):
    print(i)
    new_df = models[i].sample(10000)

0
1
2
Error: Sampling terminated. Partial results are stored in a temporary file: .sample.csv.temp. This file will be overridden the next time you sample. Please rename the file if you wish to save these results.


ValueError: Cannot convert non-finite values (NA or inf) to integer